# 1. Settings
## 1) import required libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt
import utils

### 2) Hyper parameters

In [2]:
batch_size = 16
learning_rate = 1e-4
epochs = 20

n_node = 1024
dropout_ratio = 0.5

img_size = 256

# Data Loader
### training data

In [3]:
img_dir = 'animal/train'
train_data = dset.ImageFolder(
    img_dir,
    transforms.Compose([
        transforms.CenterCrop(2 * img_size),
        transforms.RandomCrop(img_size),
        transforms.RandomHorizontalFlip(),

        transforms.Resize(img_size),
        transforms.ToTensor()
    ])
)
print(len(train_data))

train_batch = DataLoader(
    dataset=train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)

46


### Fixed dataset

In [4]:
# 2. val(Dev) data
val_dir = 'animal/val'
val_data = dset.ImageFolder(
    val_dir,
    transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.Resize(img_size),
        transforms.ToTensor()
    ])
)

val_batch = DataLoader(
    dataset=val_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

# 3. Test data
test_dir = 'animal/test'
test_data = dset.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.Resize(img_size),
        transforms.ToTensor()
    ])
)
test_batch = DataLoader(
    dataset=test_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

nclass = len(train_data.classes)
print(f'# of classes: {nclass}')
print(train_data.classes)
print(train_data.class_to_idx)
print(len(train_data))
print(f'Training: {len(train_data)}, val_data: {len(val_data)}, test_data: {len(test_data)}')

print('\nclasses')
print(train_data.classes)
print(val_data.classes)
print(test_data.classes)

# of classes: 2
['cats', 'dogs']
{'cats': 0, 'dogs': 1}
46
Training: 46, val_data: 17, test_data: 41

classes
['cats', 'dogs']
['cats', 'dogs']
['cats', 'dogs']


# 3. Model
## 1) Pretrained VGG Model

In [5]:
vgg = models.vgg19(pretrained=True)
for name, module in vgg.named_children():
    print(name)
print(list(vgg.children())[0])
print(list(vgg.children())[-1])

print(list(vgg.children())[0][0])

features
avgpool
classifier
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(

### 2) Customized Fully Model

In [6]:
base_dim = 64
fsize = img_size // 32

class MyVGG(nn.Module):
    def __init__(self):
        super(MyVGG, self).__init__()
        self.layer0 = nn.Sequential(*list(vgg.children())[0])
        self.layer1 = nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(8 * base_dim * fsize * fsize, n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout(dropout_ratio),

            nn.Linear(n_node, n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout(dropout_ratio),

            nn.Linear(n_node, n_node),
            nn.BatchNorm1d(n_node),
            nn.ReLU(),
            nn.Dropout(dropout_ratio),

            nn.Linear(n_node, nclass)
        )

        for m in self.layer1.modules():
            if isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)

    def forward(self, x):
        out = self.layer0(x),
        
        
        
        #out = out.view(out.size(0), -1)
        out = self.layer1(out)

        return out

### 3) Model on GPU

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device : {device}')

model = MyVGG().to(device)
for params in model.layer0.parameters():
    params.requires_grad = False
for params in model.layer1.parameters():
    params.requires_grad = True

for name in model.children():
    print(name)

device : cuda
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)


# 4. Optimizer & Loss

In [8]:
loss_func = nn.CrossEntropyLoss()
optimzer = optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [14]:
total_time = 0
disp_step = 10

to_train = True
if to_train == False:
    netname = './nets/catdog_vgg19_10.pkl'
    model
else:
    print(f'3 layer, n_node, {n_node}, dropout_ratio: {dropout_ratio:.2f}')
    model.eval()
    train_corr = utils.ComputeCorr(train_batch, model)
    val_corr = utils.ComputeCorr(val_batch, model)
    test_corr = utils.ComputeCorr(test_batch)
    print(f'Correc of train: {train_corr:.2f}, val: {val_corr:.2f}, test: {test_corr:.2f}')
    model.train()

    net_name = './nets/catdog_vgg19'

    # graph
    x_epoch = []
    y_train_err = []
    y_val_err = []
    y_test_err = []

    x_epoch.append(0)
    y_train_err.append(100.0 - train_corr)
    y_val_err.append(100.0 - val_corr)
    y_test_err.append(100.0 - test_corr)

    for i in range(epochs):
        start_time = time.time()
        print(f'{i+1}..')
        for img, label in train_batch:
            img = Variable(img).to(device)
            label = Variable(label).to(device)

            optimzer.zero_grad()
            output = model(img)
            loss = loss_func(output, label)
            loss.backward()
            optimzer.step()
            
        end_time = time.time()
        duration = end_time - start_time
        total_time += duration

        if i % disp_step == 0 or i == epochs-1:
            torch.save(model, net_name+f'_{i}.pkl')
            print(f'\n[{i} {epochs}] loss: {(loss.cpu()).data.numpy()}')
            
            model.eval()
            train_corr = utils.ComputeCorr(train_batch, model)
            val_corr = utils.ComputeCorr(val_batch, model)
            test_corr = utils.ComputeCorr(test_batch, model)
            
            print(f'Correct of train: {train_corr:.2f}, val: {val_corr:.2f}, test: {test_corr:.2f}')
            
            # draw graph
            x_epoch.append(i+1)
            y_train_err.append(100.0 - train_corr)
            y_val_err.append(100.0 - val_corr)
            y_test_err.append(100.0 - test_corr)
        print(f'Total time: {total_time:.2f}')
            

3 layer, n_node, 1024, dropout_ratio: 0.50
1..


AttributeError: 'tuple' object has no attribute 'flatten'